In [1]:
PORT = 5000
env = 'LOCAL' # LOCAL | ICL

In [2]:
from pathlib import Path

if env == 'LOCAL':
    WORK_DIR = Path('.')
elif env == 'ICL':
    import os
    WORK_DIR = Path('/vol/bitbucket/') / os.getenv("USER") / 'worker'

In [3]:
import uvicorn
import nest_asyncio
from fastapi import FastAPI, WebSocket, WebSocketDisconnect, HTTPException, BackgroundTasks
from fastapi.responses import JSONResponse
from fastapi.middleware.cors import CORSMiddleware
from src.constants import UI_URL, WORKER_URL
from src.models import ProcessRequest
from src.socket import Socket

from dummy_response import audit

import asyncio

nest_asyncio.apply()

app = FastAPI()

origins = [
    str(UI_URL),
    str(WORKER_URL),
]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# TODO: If the UI never connects persist the result for sometime
# and return it when the UI tries to connect
async def process_work(data: dict):
    await asyncio.sleep(0.5)
    await Socket.send({"status": "building_dt"})
    await asyncio.sleep(0.5)

    # Simulate work processing
    steps = 5
    for i in range(steps):
        await asyncio.sleep(0.5)
        await Socket.send({
            "status": "analyzing",
            "progress": {"current": i+1, "total": steps}
        })

    await Socket.send({
        "status": "complete",
        "done": True,
        "result": audit
    })
    await Socket.close()

@app.post("/process")
async def process(request: ProcessRequest, background_tasks: BackgroundTasks):
    Socket.register_incoming_id(request.socket)
    background_tasks.add_task(process_work, request)
    return JSONResponse(content={"status": 200})

@app.websocket("/ws/{socket_id}")
async def websocket_endpoint(websocket: WebSocket, socket_id: str):
    async with Socket(websocket, socket_id) as connection:
        await connection.listen()


if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=PORT)

INFO:     Started server process [53118]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:5000 (Press CTRL+C to quit)


INFO:     127.0.0.1:63030 - "POST /process HTTP/1.1" 200 OK


INFO:     ('127.0.0.1', 63032) - "WebSocket /ws/b037ac6e-bce2-4d01-902a-b8d451d4f01e" [accepted]
INFO:     connection open
INFO:     connection closed


Socket b037ac6e-bce2-4d01-902a-b8d451d4f01e disconnected
INFO:     127.0.0.1:63080 - "POST /process HTTP/1.1" 200 OK


INFO:     ('127.0.0.1', 63082) - "WebSocket /ws/77811fa0-f48c-43eb-bc7d-683ea3b045da" [accepted]
INFO:     connection open
INFO:     connection closed


Socket 77811fa0-f48c-43eb-bc7d-683ea3b045da disconnected
